1. 阻塞式IO
2. 非阻塞式IO
3. IO多路复用
4. 信号驱动式IO （用的少）
5. 异步IO

1. 阻塞式：普通的代码，普遍都是这种阻塞式的，即代码运行到耗时的地方会等待返回再继续往下执行。

2. 非阻塞式：在处理一些耗时的代码时，将其设置成立马返回一个结果的形式，然后后台还在继续执行那个代码，但是主代码中往下执行时需要对这个立马返回的结果进行状态判断，使用while一直轮询判断其状态是否完成，如果完成了就往下继续执行主代码。举例：socket的连接本身是阻塞式的，但是可以通过client.setblocking(False)设置为非阻塞式，如果在连接的下一行主代码中是立马使用这个连接的实例的话，那么while轮询要写在其前面一直判断其状态，这样的话其实和阻塞式是相差无几的。但是如果在连接的下一行主代码是其他一些稍微耗时的计算代码时，那么这个非阻塞式就很有必要了，因为它不需要等待连接建立完成就可以继续往下执行这个计算代码，只需在调用这个连接的代码前面使用while轮询一下它的状态就行了。

3. 复用：其实复用和上面的非阻塞式很像，只不过上面是通过while来轮询一个连接而已，复用是使用select来监听多个连接，一旦发现哪个连接完成建立了就立马处理这个连接。

注意：阻塞和非阻塞其实是相对CPU有没有在被这个程序的代码使用来说的，像等待网页返回这种就是阻塞式，因为CPU没有被使用了，它要等到返回才能继续往下执行。非阻塞是指其通过while一直轮询，造成CPU一直不会处于空闲状态，使得这个程序的代码会一直在运行，所以称为非阻塞。
阻塞->CPU闲下来了，
非阻塞->CPU忙起来

### 回调之痛（IO多路复用）

1. 异常怎么处理
2. 多层回调咋办，代码难以维护
3. 嵌套多层回调，其中某一环节出问题怎么定位
4. 如果有数据需要被每个回调处理怎么办
5. 怎么使用当前函数的局部变量

### 协程背景

In [3]:
# C10K（C代表client，所以是单机处理（保持）10000个并发链接问题）
# C10M（C代表client，M是兆，等于100万，所以是单机处理（保持）1000万并发链接问题）

1. 回调模式编码复杂度高
2. 同步编程的并发性不高
3. 多线程编程需要线程间同步，同步需要用到锁lock，使用锁会使得线程并发性降低。

引出问题：
1. 采用同步的方式去编写异步的代码
2. 使用单线程去切换任务：
    1. 线程是由操作系统切换的，单线程切换意味着我们需要程序员自己去调动任务
    2. 不在需要锁，并发性高，如果单线程内切换函数，性能远高于线程切换，并发现更高

> 1. 为解决上面两个问题，协程应运而生
> 2. 理解：协程 -> 有多个入口的函数， 可以暂停的函数， 我比较倾向理解为可以暂停的函数(可以向暂停的地方传入值)

### 生成器

In [5]:
# print(gen.send(html)) #send方法可以传递值进入生成器内部，同时还可以重启生成器执行到下一个yield位置
# 1.启动生成器方式有两种， next(), send
# 在调用send发送非none值之前，我们必须启动一次生成器， 方式有两种1. gen.send(None), 2. next(gen)

In [ ]:
# def get_url(url):
#     #do someting 1
#     html = get_html(url) #此处暂停，切换到另一个函数去执行
#     # #parse html
#     urls = parse_url(html)
#
# def get_url(url):
#     #do someting 1
#     html = get_html(url) #此处暂停，切换到另一个函数去执行
#     # #parse html
#     urls = parse_url(html)

#传统函数调用 过程 A->B->C
#我们需要一个可以暂停的函数，并且可以在适当的时候恢复该函数的继续执行
#出现了协程 -> 有多个入口的函数， 可以暂停的函数， 可以暂停的函数(可以向暂停的地方传入值)

def gen_func():
    #1. 可以产出值， 2. 可以接收值(调用方传递进来的值)
    html = yield "http://projectsedu.com"
    print(html)
    return "bobby"

#1. throw, close



#1. 生成器不只可以产出值，还可以接收值


if __name__ == "__main__":
    gen = gen_func()
    #在调用send发送非none值之前，我们必须启动一次生成器， 方式有两种1. gen.send(None), 2. next(gen)
    url = gen.send(None)
    #download url
    html = "bobby"
    print(gen.send(html)) #send方法可以传递值进入生成器内部，同时还可以重启生成器执行到下一个yield位置
    print(gen.send(html))
    #1.启动生成器方式有两种， next(), send

    # print(next(gen))
    # print(next(gen))
    # print(next(gen))
    # print(next(gen))



In [9]:
from itertools import chain

my_list = [1,2,3]
my_dict = {
    "bobby1":"http://projectsedu.com",
    "bobby2":"http://www.imooc.com",
}

for value in chain(my_list, my_dict, range(5, 10)):  # 将多个可迭代对象连接在一起使用
    print(value)

1
2
3
bobby1
bobby2
5
6
7
8
9


### yield from

In [26]:
final_result = {}

def sales_sum(pro_name):
    total = 0
    nums = []
    while True:
        x = yield
        print(pro_name+"销量: ", x)
        if not x:
            break
        total += x
        nums.append(x)
    return total, nums


def middle(key):
    final_result[key] = yield from sales_sum(key)# yield from 是直接和调用方建立通道的，绕过了委托生成器
    print(key+"销量统计完成！！.")
    yield  # 暂停函数 如果没有暂停函数的话，委托生成器也会执行完毕从而报错

def main():
    data_sets = {
        "bobby牌面膜": [1200, 1500, 3000],
        "bobby牌手机": [28,55,98,108 ],
        "bobby牌大衣": [280,560,778,70],
    }
    for key, data_set in data_sets.items():
        print("start key:", key)
        m = middle(key)   # 这是生成器，不是函数调用  第二次实例化使得前一次的暂停函数无用了
        m.send(None) # 预激middle协程
        for value in data_set:
            m.send(value)   # 给协程传递每一组的值
        m.send(None)
    print("final_result:", final_result)

if __name__ == '__main__':
    main()

start key: bobby牌面膜
bobby牌面膜销量:  1200
bobby牌面膜销量:  1500
bobby牌面膜销量:  3000
bobby牌面膜销量:  None
bobby牌面膜销量统计完成！！.
start key: bobby牌手机
bobby牌手机销量:  28
bobby牌手机销量:  55
bobby牌手机销量:  98
bobby牌手机销量:  108
bobby牌手机销量:  None
bobby牌手机销量统计完成！！.
start key: bobby牌大衣
bobby牌大衣销量:  280
bobby牌大衣销量:  560
bobby牌大衣销量:  778
bobby牌大衣销量:  70
bobby牌大衣销量:  None
bobby牌大衣销量统计完成！！.
final_result: {'bobby牌面膜': (5700, [1200, 1500, 3000]), 'bobby牌手机': (289, [28, 55, 98, 108]), 'bobby牌大衣': (1688, [280, 560, 778, 70])}


In [28]:
def gen_fun():
    html = yield "http://www.baidu.com"  # 先返回再接收值的
    print("gen：", html)
    yield 1  # 如果这里不写yield的话，则会报异常StopIteration，因为它认为这个已经不是生成器了，因为没有了yield，前面那个yield已经被使用了
    
if __name__ == "__main__":
    gen = gen_fun()
    url = gen.send(None)  # 等价 url = next(gen) 这里传None是预激生成器，预激也会直接定位到生成器的yield中并返回yield出去的值，然后暂停住
    print("main：", url)
    s = gen.send("haha")
    print(s)

main： http://www.baidu.com
gen： haha
1


In [31]:
def sales_sum(pro_name):
    total = 0
    nums = []
    while True:
        x = yield
        print(pro_name+"销量: ", x)
        if not x:
            break
        total += x
        nums.append(x)
    return total, nums

if __name__ == '__main__':
    my_gen = sales_sum("haha")
    my_gen.send(None) # 预激
    my_gen.send(1000)
    my_gen.send(2000)  # 生成器中是while True按理说是一直循环的，可是因为是有yield字段
    # 它会暂停函数，又由于主函数中运行结束了，所以子函数自然而然要跟着结束呀.
#     my_gen.send(None) # 执行这行代码会报错StopIteration，因为如果传None过去的话
    # 就跳出了while True循环，说明完全消耗完了yield关键字，那么默认会调用close关闭
    # 生成器，接着抛出异常StopIteration。

haha销量:  1000
haha销量:  2000
haha销量:  None


StopIteration: 